# Лабораторна робота №1

## Завдання

Розв'язати задачу знаходження ваг із МПП і зробити висновки щодо альтернатив. Визначити узгодженість МПП за відповідним коефіцієнтом, порівняти його з порогами і зробити висновок про узгодженість матриці. Порівняти результати, отримані різними методами.

Методи обчислення ваг:
- AN
- RGMM

Показники узгодженості:
- HCR(розрахувати HRCI)
- GCI

Розглядається задача №2:
> Нехай задача полягає в оцінюванні наступних чотирьох варіантів вкладення коштів: придбання акцій, оформлення депозиту, придбання облігацій, придбання дорогоцінних металів за критерієм «надійність вкладення коштів». За результатами парних порівнянь цих варіантів встановлено, що другий варіант ненабагато кращий за перший і третій варіанти і суттєво кращий за четвертий, перший варіант має однакову надійність, що і третій, і ненабагато кращий за четвертий варіант, перевага третього варіанту над четвертим – між слабкою і суттєвою.

## Виконання роботи

### Імпорт бібліотек

In [1]:
%pylab inline
import seaborn as sns
import scipy as sp

Populating the interactive namespace from numpy and matplotlib


In [2]:
from ipy_table import make_table as html_matrix

### Генерація таблиці

#### Генератори випадкових мультиплікативних/адитивних матриць попарних порівнянь заданого розміру

In [3]:
def rand_mult_pw_val(size):
    res = triu(random.randint(-9, 10, (size, size)), 1).astype(float64)
    res[res < 0] = 1 / abs(res[res < 0])
    res += tril(1 / res.T, -1) + identity(size)
    return res

def rand_add_pw_val(size):
    res = triu(random.randint(-9, 10, (size, size)), 1).astype(float64)
    res -= res.T
    return res

#### Фіксація матриці з умови задачі

In [4]:
given_matrix = array([
        [  1, 1/3,   1, 3],
        [  3,   1,   3, 5],
        [  1, 1/3,   1, 4],
        [1/3, 1/5, 1/4, 1]
    ])
print('Задана матриця:')
html_matrix(given_matrix)

Задана матриця:


1.0,0.333333333333,1.0,3.0
3.0,1.0,3.0,5.0
1.0,0.333333333333,1.0,4.0
0.333333333333,0.2,0.25,1.0


### Обчислення ваг

#### 1. AN (additive normalization)

Нехай $s_j = \sum\limits_{i=1}^n d_{ij}$ – сума j-го стовпчика заданої експертами МПП.

Тоді вагами є величини $(s_1^{-1}, \dots , s_n^{-1})$ обернені до сум стовпчиків заданої матриці.

In [5]:
s = given_matrix.sum(axis=0)
weights_an = 1/s
print('Ваги, розраховані з методу AN:')
print(weights_an)

Ваги, розраховані з методу AN:
[ 0.1875      0.53571429  0.19047619  0.07692308]


#### 2. RGMM (row geometic mean method)

Під час розрахунку ваг методом RGMM використовується наступна формула:
$$
v_i = \sqrt[n]{\prod\limits_{j=1}^{n} d_{ij}}
$$
Тоді нормовані до 1 ваги альтернатив розраховуються
$$
w_i = \sqrt[n]{\prod\limits_{j=1}^{n} d_{ij}} \; \big/ \; \sum\limits_{i=1}^n \sqrt[n]{\prod\limits_{j=1}^{n} d_{ij}}
$$

In [6]:
v_i = sp.stats.gmean(given_matrix, axis=1)
print('Ненормовані ваги, розраховані методом RGMM')
print(v_i, '\n')
weights_rgmm = v_i / sum(v_i)
print('Нормовані ваги, розраховані методом RGMM')
print(weights_rgmm)

Ненормовані ваги, розраховані методом RGMM
[ 1.          2.59002006  1.07456993  0.35930411] 

Нормовані ваги, розраховані методом RGMM
[ 0.19904878  0.51554034  0.21389184  0.07151905]


### Оцінка узгодженості

#### 1. HCR (з розрахунком HRCI)

Показник HCR як правило застосовується для перевірки коректності застосування методу [AN][1].
[1]: #1.-AN-(additive-normalization)

Гармонічним індексом узгодженості (harmonic consistency index, HCI) називається
$$
HCI(n) = \frac{(HM(s)-n)(n+1)}{n(n-1)}
$$
де $HM(s)=n \left(\sum\limits_{j=1}^n s_j^{-1} \right)^{-1}$ – гармонічне середнє для сум по стовпчикам вихідної матриці.

Гармнонічним відношенням узгодженості (harmonic consistency relation) називається
$$
HCR(n) = \frac{HCI(n)}{HRCI(n)}
$$
де $HRCI(n)$ – середнє значення $HCI(n)$ для випадкових МПП.

<table>
<tr>
<th colspan="3">Порогове значення HCR</th>
</tr>
<tr>
<th>$n=3$</th><th>$n=4$</th><th>$n\geq5$</th>
</tr>
<tr>
<td>0.05</td><td>0.08</td><td>0.1</td>
</tr>
</table>

In [5]:
def calculate_hci(mat):
    s_j = mat.sum(axis=0)
    hm = sp.stats.hmean(s_j)
    n = len(s_j)
    return (hm - n) * (n + 1) / n / (n - 1)

In [6]:
size = 4
count = 1000000
hrci = mean(list(calculate_hci(rand_mult_pw_val(size)) for i in range(count)))
print("Розрахований HRCI(4):", hrci)

Розрахований HRCI(4): 0.956236245021


In [9]:
hcr = calculate_hci(given_matrix) / hrci
print("Узгодженість вихідної матриці за методом HCR:", hcr)

Узгодженість вихідної матриці за методом HCR: 0.0165404801513


Оскільки 0.0165404801513 < 0.08, то можна вважати вихідну матрицю консистентною за показником HCR.

#### 2. GCI (geometric consistency index)

Показник GCI як правило застосовується для перевірки коректності застосування методу [RGMM][1].
[1]: #2.-RGMM-(row-geometic-mean-method)

Показник узгодженості GCI вираховується за формулою
$$GCI=\frac{2}{(n-1)(n-2)} \sum\limits_{i < j} \log ^2 e_{ij}$$
де $e_{ij}=d_{ij} \frac{v_j}{v_i}$ – помилка апроксимації відношення ваг $\frac{v_i}{v_j}$ за допомогою елемента МПП $d_{ij}$; $v_i = \sqrt[n]{\prod\limits_{j=1}^{n} d_{ij}}$.

<table>
<tr>
<th colspan="3">Порогове значення GCI</th>
</tr>
<tr>
<th>$n=3$</th><th>$n=4$</th><th>$n=5$</th>
</tr>
<tr>
<td>0.1573</td><td>0.3526</td><td>0.370</td>
</tr>
</table>

In [10]:
e_ij = multiply(given_matrix, v_i / v_i[:, newaxis])
lg_e2 = log(tril(e_ij, -1))**2
n = len(v_i)
gci = lg_e2[lg_e2 != inf].sum() * 2 / (n - 1) / (n - 2)
print("Узгодженість вихідної матриці за методом GCI:", gci)

Узгодженість вихідної матриці за методом GCI: 0.0995583040276


Оскільки 0.0995583040276 < 0.3526, то можна вважати вихідну матрицю консистентною за показником GCI.

## Висновки

Під час виконання лабораторної роботи було проведено оцінку важливостей 4 заданих альтернатив вкладення грошей. Використовувалися методи AN та RGMM. В результаті отримали наступні ваги:

| Метод |    $w_1$   |    $w_2$   |    $w_3$   |    $w_4$   |
|:-----:|:----------:|:----------:|:----------:|:----------:|
|   AN  |   0.1875   | 0.53571429 | 0.19047619 | 0.07692308 |
|  RGMM | 0.19904878 | 0.51554034 | 0.21389184 | 0.07151905 |

Нескладно помітити, що отримані різними методами ваги досить сильно корелюють. Таким чином, виходячи з умови, отримали, що найкращою альтернативою є друга – **оформлення депозиту**.

Після виконання оцінки ваг, було перевірено консистентність матриці показниками, відповідними до застосованих методів.

| Показник | Порогове значення при $n = 4$ | Значення для заданої матриці |
|:--------:|:-----------------------------:|:----------------------------:|
|    HCR   |              0.08             |        0.0165404801513       |
|    GCI   |             0.3526            |        0.0995583040276       |

Таким чином, вихідна матриця є консистентною за обома показниками.